In [5]:
import json

# Open the JSON file
with open('./../Poems_And_Outputs/Poems_Unparsed.JSON', 'r', encoding='utf-8') as f:
    data = json.load(f)

songs = []
labels = []

for song in data:
  if not song['year'] == "":
    songs.append(song['content'])
    if int(song['year']) < 1900:
      labels.append(0)
    elif int(song['year']) < 1910:
      labels.append(1)
    else:
      labels.append(2)

In [6]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Embedding ,Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Sample data
hebrew_songs = songs

decade_labels = labels

# Tokenize words
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(hebrew_songs)
sequences = tokenizer.texts_to_sequences(hebrew_songs)

# Padding sequences
max_len = max(len(seq) for seq in sequences)
padded_sequences = tf.keras.preprocessing.sequence.pad_sequences(sequences, maxlen=max_len, padding='post')

# Convert to numpy arrays
X = np.array(padded_sequences)
y = np.array(decade_labels)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Regular neural network model
model = Sequential([
    Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=100, input_length=max_len),
    Flatten(),
     *[Dense(128, activation='relu') for _ in range(50)],
    Dropout(0.5),
    Dense(3, activation='softmax')  # Adjusted for 3 labels
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Accuracy:", accuracy)


ModuleNotFoundError: No module named 'tensorflow'